Importing libraries which are required for Webscrapping

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

# DATA COLLECTION : WebScrapping

 - problem statement : Finding a desired properties in the affordable price.
 - Web scraping is a method to obtain large amounts of data from websites.

In [2]:
%%time
titles = []
types = []
sft = []
prices = []
status = []
handover = []
location = []
seller_info = []
Cost = []
BHK = []
colony = []
# 2357 pages
for i in range(1,60):
    URL = 'https://www.makaan.com/hyderabad-residential-property/buy-property-in-hyderabad-city?page={}'. format(i)
    
    page = requests.get(URL)
    htmlCode = page.text
    
    soup = BeautifulSoup(htmlCode)
    
    for container in soup.find_all('div', attrs={'class' : 'cardLayout clearfix'}):

        title = container.find('div', attrs={'class' : 'title-line'})
        if title is None:
            titles.append(np.NaN)
        else:
            titles.append(title.text)

        squarefeet = container.find('td', attrs={'class' : 'size'})
        if squarefeet is None:
            sft.append(np.NaN)
        else:
            sft.append(int(squarefeet.text))
            
        price = container.find('td', attrs={'class' : 'price'})
        if price is None:
            prices.append(np.NaN)
        else:
            if price.text.split()[-1] == "Cr":
                    c = price.text.split()[0]
                    d = float(c)
                    b = ("%.7f"% d)
                    e = str(b)
                    f = e.replace(".","")
                    prices.append(int(f))
            elif price.text.split()[-1] == "L":
                    c = price.text.split()[0]
                    d = float(c)
                    b = ("%.5f"% d)
                    e = str(b)
                    f = e.replace(".","")
                    prices.append(int(f))

        const_status = container.find('td', attrs={'class' : 'val'})
        if const_status is None:
            status.append(np.NaN)
        else:
            status.append(const_status.text)
            
        hand_over = container.find('ul', attrs={'class' : 'listing-details'})
        if hand_over is None:
            handover.append(np.NaN)
        elif "by" in hand_over.text:
                s = re.findall(r"(\w+\s\d{4})",hand_over.text)
                for i in s:
                    handover.append(i)
        elif "old" in hand_over.text:
                s = re.findall(r"(\d+\s\w+\sold)",hand_over.text)
                for i in s:
                    handover.append(i)
        else:
            handover.append(np.NaN)
            
        area = container.find('div', attrs={'class' : 'locWrap'})
        if area is None:
            location.append(np.NaN)
        else:
            location.append(area.text.split(",")[0])
            
        sellerinfo = container.find('div', attrs={'class' : 'seller-info'})
        if sellerinfo is None:
            seller_info.append(np.NaN)
        else:
            seller_info.append(sellerinfo.text)
            
        persft = container.find('td', attrs={'class' : "lbl rate"})
        if persft is None:
            Cost.append(np.NaN)
        else:
            Cost.append(int(persft.text.split("/")[0].replace(",","")))
            
        bhk = container.find('span', attrs={'class' : "val"})
        if bhk is None:
            BHK.append(np.NaN)
        else:
            a = bhk.text
            b = float(a)
            BHK.append(int(b))
            
            

    title = soup.find_all('div', attrs={'class' : 'title-line'})
    for container in title:
        if container is None:
            types.append(np.NaN)
        else:
            if container.text.split()[1] == "BHK":
                types.append(container.text.split()[2])
            elif (len(container.text.split()) > 2) and (container.text.split()[2] == "in"):
                types.append(container.text.split(" in")[0])
            else:
                types.append(container.text)
                
    title = soup.find_all('div', attrs={'class' : 'title-line'})
    for container in title:
        if container is None:
            colony.append(np.NaN)
        elif "in" in container.text.split():
                colony.append(container.text.split(" in")[1])
        else:
                colony.append(np.NaN)
    

Wall time: 1min 29s


In [3]:
len(handover)

1180

Converting the scrapped data into dataframe using pandas

In [125]:
d = {"Titles" : titles ,"Type" : types ,"BHK": BHK ,"sft" : sft ,"Costpersft" : Cost, "Prices" : prices ,"construction_status" : status  ,"colony" : colony , "Location" : location , "seller_info" : seller_info , "Handover" : handover}

In [126]:
df = pd.DataFrame(d)

In [127]:
df

,Titles,Type,BHK,sft,Costpersft,Prices,construction_status,colony,Location,seller_info,Handover
0,3 BHK Apartment in Lalitha Divine County,Apartment,3,1535,4900,7521000,Under Construction,Lalitha Divine County,Ameenpur,Lalitha Construction and DevelopersBUILDER0,Jan 2023
1,3 BHK Villa in Nest Bliss In The Woods,Villa,3,4000,8275,33100000,Under Construction,Nest Bliss In The Woods,Harshaguda,Nest MakersBUILDER0,Sep 2024
2,3 BHK Apartment in Candeur Crescent,Apartment,3,2548,7810,19900000,Under Construction,Candeur Crescent,Serilingampally,Candeur Developers And BuildersBUILDER0,Dec 2025
3,3 BHK Apartment in GHR Callisto,Apartment,3,1775,5199,9228000,Under Construction,GHR Callisto,Kollur,GHR InfraBUILDER0,Nov 2025
4,3 BHK Apartment in Vazhraa Vazhraa Prathik,Apartment,3,1835,6500,11900000,Under Construction,Vazhraa Vazhraa Prathik,Nizampet,Vazhraa Nirmaan Pvt LtdBUILDER0,Apr 2023
...,...,...,...,...,...,...,...,...,...,...,...
1175,10 BHK Independent House,Independent,10,9000,7222,65000000,Ready to move,NaN,Madhapur,Sahara Real Estate ConsultantsAGENT0,NaN
1176,3 BHK Apartment,Apartment,3,1670,10000,16700000,Ready to move,NaN,Madhapur,Sahara Real Estate ConsultantsAGENT0,NaN
1177,3 BHK Apartment,Apartment,3,1550,10004,15500000,Ready to move,NaN,Madhapur,Sahara Real Estate ConsultantsAGENT0,NaN
1178,3 BHK Apartment,Apartment,3,1360,6617,9000000,Ready to move,NaN,Madhapur,Sahara Real Estate ConsultantsAGENT0,NaN


We converted Dataframe into csv file

In [ ]:
df.to_csv("HYD_Cities")